In [ ]:
import pandas as pd

import warnings

warnings.filterwarnings(action='ignore')



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dacon_df = pd.read_csv('/content/drive/MyDrive/khuda_team_project/dacon/train.csv')
gpt_df = pd.read_csv('/content/drive/MyDrive/khuda_team_project/chat-gpt/chatgpt-reddit-comments.csv')

In [ ]:
dacon_df.head(3)

,ID,Utterance,Speaker,Dialogue_ID,Target
0,TRAIN_0000,also I was the point person on my company’s tr...,Chandler,0,neutral
1,TRAIN_0001,You must’ve had your hands full.,The Interviewer,0,neutral
2,TRAIN_0002,That I did. That I did.,Chandler,0,neutral


In [ ]:
dacon_df = dacon_df.drop(['ID', "Speaker", 'Dialogue_ID'], axis=1)
dacon_df.head(3)

,Utterance,Target
0,also I was the point person on my company’s tr...,neutral
1,You must’ve had your hands full.,neutral
2,That I did. That I did.,neutral


In [ ]:
dacon_df[dacon_df['Target']=="disgust"]

,Utterance,Target
27,Just coffee! Where are we gonna hang out now?,disgust
105,Oh my god. You are pure evil.,disgust
125,"That’s great, but shouldn’t you be on the toil...",disgust
127,What’s wrong with you?,disgust
133,That’s a hernia.,disgust
...,...,...
9889,"Fine. Okay, enjoy your concert.",disgust
9976,"They were huge. When she sneezed, bats flew ou...",disgust
9978,"I'm tellin' you, she leaned back; I could see ...",disgust
9982,"When I first moved to the city, I went out a c...",disgust


-------

In [ ]:
gpt_df.head(3)

,Unnamed: 0,comment_id,comment_parent_id,comment_body,subreddit
0,0,iztdxuh,t3_zj2aeu,"I've been shocked for days now, I don't need c...",r/ChatGPT
1,1,iztn0q0,t3_zj2aeu,\n\nI am so angry right now. I just wasted my...,r/ChatGPT
2,2,izudrph,t3_zj2aeu,chatgpt karma whoring is here folks! just when...,r/ChatGPT


In [ ]:
gpt_df = gpt_df.drop(['Unnamed: 0', 'comment_id', "comment_parent_id", 'subreddit'], axis=1)
gpt_df.head(3)

,comment_body
0,"I've been shocked for days now, I don't need c..."
1,\n\nI am so angry right now. I just wasted my...
2,chatgpt karma whoring is here folks! just when...


In [ ]:
gpt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52416 entries, 0 to 52415
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   comment_body  52415 non-null  object
dtypes: object(1)
memory usage: 409.6+ KB


-----------

In [ ]:
dacon_df['Target'].value_counts()

neutral     4710
joy         1743
surprise    1205
anger       1109
sadness      683
disgust      271
fear         268
Name: Target, dtype: int64

In [ ]:
print(dacon_df.shape, gpt_df.shape)

(9989, 2) (52416, 1)


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(dacon_df['Utterance'].tolist())

len(tokenizer1.word_index)

5921

In [ ]:
tokenized_dacon_text = tokenizer1.texts_to_sequences(dacon_df['Utterance'].tolist())

In [ ]:
len_list1 = []
for elem in tokenized_dacon_text:
    len_list1.append(len(elem))

max(len_list1)

69

In [ ]:
x = 0
y = 0
z = 0
over = 0

for elem in len_list1:
    if elem<20:
        x+=1
    elif 10<=elem<40:
        y+=1
    elif 20<=elem<60:
        z+=1
    else:
        over+=1

print(x, y, z, over)

9392 590 6 1


In [ ]:
tokenizer2 = Tokenizer()
tokenizer2.fit_on_texts(gpt_df['comment_body'].astype('str').tolist())

len(tokenizer2.word_index)

46494

In [ ]:
tokenized_gpt_text = tokenizer1.texts_to_sequences(gpt_df['comment_body'].astype('str').tolist())

len_list2 = []
for elem in tokenized_gpt_text:
    len_list2.append(len(elem))

max(len_list2)

1071

In [ ]:
x = 0
y = 0
z = 0
over = 0

for elem in len_list2:
    if elem<20:
        x+=1
    elif 10<=elem<40:
        y+=1
    elif 20<=elem<60:
        z+=1
    else:
        over+=1

print(x, y, z, over)

28184 11715 5090 7427


In [ ]:
max_sequence = 64

-----------------

In [ ]:
!pip install keras-bert
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
X = dacon_df['Utterance'].values
y = dacon_df['Target'].values

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=7)

def get_model():
    input_ids = tf.keras.layers.Input(shape=(64,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.layers.Input(shape=(64,), dtype=tf.int32, name="attention_mask")
    outputs = model([input_ids, attention_mask])
    bert_model = tf.keras.models.Model(inputs=[input_ids, attention_mask], outputs=outputs)
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
    bert_model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    return bert_model

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

# 학습 데이터 전처리
def preprocess_data(texts, tokenizer, max_len=64):
    input_ids = []
    attention_masks = []
    
    for text in texts:
        encoded_dict = tokenizer.encode_plus(
                            text,                      # 문장 하나씩 가져와 인코딩
                            add_special_tokens = True, # CLS, SEP 토큰 추가
                            max_length = max_len,      # 최대 시퀀스 길이
                            pad_to_max_length = True,  # 최대 길이에 따라 패딩 추가
                            return_attention_mask = True, # attention mask 생성
                            return_tensors = 'tf',     # 텐서플로우 형태로 반환
                       )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
        
    return np.concatenate(input_ids, axis=0), np.concatenate(attention_masks, axis=0)

# 타겟 데이터 라벨 인코딩
def encode_labels(labels):
    encoder = LabelEncoder()
    encoder.fit(labels)
    return encoder.transform(labels)
    
X_train, attention_mask_train = preprocess_data(X, tokenizer)
y_train = encode_labels(y)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
model = get_model()
model.fit([X_train, attention_mask_train], y_train, epochs=10, batch_size=32)